# Exploratory Data Analysis (EDA)

This notebook contains exploratory data analysis for the two datasets:
1. Power Combined Dataset
2. HPC Kernel Events Dataset

We'll analyze the datasets separately and then look at potential ways to combine their features.